In [ ]:
# INSTALLS
# llama-index-readers-file pymupdf
# pip install llama-index-llms-azure-openai
# ipykernel
# pip install llama-index-llms-ollama

# NOT YET
# %pip install llama-index-embeddings-huggingface

# https://docs.llamaindex.ai/en/stable/examples/low_level/oss_ingestion_retrieval/

In [ ]:
# Imports
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.ollama import Ollama

In [ ]:
# api_key = "<api-key>"
# azure_endpoint = "https://<your-resource-name>.openai.azure.com/"
# api_version = "2023-07-01-preview"

# llm = AzureOpenAI(
#     model="gpt-35-turbo-16k",
#     deployment_name="my-custom-llm",
#     api_key=api_key,
#     azure_endpoint=azure_endpoint,
#     api_version=api_version,
# )
llm = Ollama(model="llama2", request_timeout=60.0)

In [4]:
# Data load
loader = PyMuPDFReader()
documents = loader.load(file_path="backend/rag_data/llama2.pdf")

In [ ]:
node_parser = SentenceSplitter(chunk_size=1024)
nodes = node_parser.get_nodes_from_documents(documents)
index = VectorStoreIndex(nodes)
query_engine = index.as_query_engine(llm=llm)

In [ ]:
# Test
query_str = (
    "What is the specific name given to the fine-tuned LLMs optimized for"
    " dialogue use cases?"
)

generated_answer = str(query_engine.query(query_str))


In [ ]:
# DATASET GENERATION